[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/proxy-statement-def14a-python.ipynb)

# Parse SEC EDGAR Proxy Statements (DEF 14A) with Python -- Free, No API Key

Use **edgartools** to parse SEC proxy statements (DEF 14A) in Python -- completely free, no API key or paid subscription required. Proxy statements contain executive pay, pay-vs-performance data, and corporate governance disclosures that every public company must file annually.

**What you'll learn:**
- Parse a DEF 14A proxy statement into a structured Python object
- Get CEO and executive compensation with 5-year history
- Extract pay-vs-performance data (TSR, net income, company KPIs)
- See which performance measures drive executive pay
- Compare CEO pay across companies
- Access governance indicators

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Parse a Proxy Statement in 2 Lines

Every public company files a DEF 14A proxy statement before its annual meeting. edgartools parses the XBRL data into a structured object with compensation, performance, and governance data:

In [ ]:
proxy = Company("AAPL").get_filings(form="DEF 14A")[0].obj()
proxy

## 5-Year Executive Compensation History

Proxy statements include a summary compensation table covering the past 5 fiscal years. Get it as a pandas DataFrame:

In [ ]:
ec = proxy.executive_compensation

display = ec.copy()
display["fiscal_year_end"] = pd.to_datetime(display["fiscal_year_end"]).dt.strftime("%Y")
for col in ["peo_total_comp", "peo_actually_paid_comp", "neo_avg_total_comp", "neo_avg_actually_paid_comp"]:
    display[col] = display[col].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else "N/A")
display.columns = ["Year", "CEO Total", "CEO Actually Paid", "NEO Avg Total", "NEO Avg Paid"]
display

## Pay vs Performance

The SEC requires companies to show how executive pay relates to company performance. This includes total shareholder return (TSR), peer group TSR, and net income over 5 years:

In [ ]:
pvp = proxy.pay_vs_performance

display = pvp[["fiscal_year_end", "peo_actually_paid_comp", "total_shareholder_return", "peer_group_tsr", "net_income"]].copy()
display["fiscal_year_end"] = pd.to_datetime(display["fiscal_year_end"]).dt.strftime("%Y")
display["peo_actually_paid_comp"] = display["peo_actually_paid_comp"].apply(lambda x: f"${x/1e6:,.1f}M")
display["net_income"] = display["net_income"].apply(lambda x: f"${x/1e9:,.1f}B")
display["total_shareholder_return"] = display["total_shareholder_return"].apply(lambda x: f"{x}%")
display["peer_group_tsr"] = display["peer_group_tsr"].apply(lambda x: f"{x}%")
display.columns = ["Year", "CEO Paid", "Company TSR", "Peer TSR", "Net Income"]
display

## Performance Measures and Governance

Companies must disclose which performance measures drive executive compensation, plus governance indicators like insider trading policies:

In [ ]:
print(f"Company Selected Measure: {proxy.company_selected_measure}")
print(f"Selected Measure Value:   ${proxy.company_selected_measure_value:,.0f}")
print(f"\nAll Performance Measures:")
for measure in proxy.performance_measures:
    print(f"  - {measure}")
print(f"\nInsider Trading Policy:   {'Adopted' if proxy.insider_trading_policy_adopted else 'Not disclosed'}")

## Compare CEO Pay Across Companies

Pull proxy data from multiple companies to compare CEO compensation, shareholder returns, and what drives pay:

In [ ]:
tickers = ["AAPL", "MSFT", "NVDA", "JPM"]
rows = []

for ticker in tickers:
    proxy = Company(ticker).get_filings(form="DEF 14A")[0].obj()
    rows.append({
        "Ticker": ticker,
        "CEO": str(proxy.peo_name),
        "CEO Total Pay": f"${proxy.peo_total_comp:,.0f}" if proxy.peo_total_comp else "N/A",
        "Company TSR": f"{proxy.total_shareholder_return}%" if proxy.total_shareholder_return else "N/A",
        "Peer TSR": f"{proxy.peer_group_tsr}%" if proxy.peer_group_tsr else "N/A",
        "Key Metric": str(proxy.company_selected_measure),
    })

pd.DataFrame(rows).set_index("Ticker")

## Why EdgarTools?

EdgarTools is free and open-source. Compare parsing SEC proxy statements:

**With edgartools (free, no API key):**
```python
proxy = Company("AAPL").get_filings(form="DEF 14A")[0].obj()
proxy.peo_total_comp               # CEO total pay
proxy.pay_vs_performance            # 5-year pay vs TSR DataFrame
proxy.performance_measures          # KPIs driving pay
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import ExtractorApi
api = ExtractorApi(api_key="YOUR_PAID_API_KEY")
# ... find DEF 14A URL, download, parse XBRL manually,
# ... map ecd: namespace concepts by hand
```

With edgartools, proxy statement data is parsed from XBRL automatically -- compensation history, pay-vs-performance, and governance all accessible instantly.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Parse a proxy statement ──
proxy = Company("AAPL").get_filings(form="DEF 14A")[0].obj()

# ── CEO compensation ──
proxy.peo_name                       # CEO name
proxy.peo_total_comp                 # Total from summary comp table
proxy.peo_actually_paid_comp         # Compensation actually paid
proxy.neo_avg_total_comp             # Average NEO total

# ── 5-year history (DataFrames) ──
proxy.executive_compensation         # Comp table: 5 years
proxy.pay_vs_performance             # Pay vs TSR/income: 5 years

# ── Performance and governance ──
proxy.total_shareholder_return       # Company TSR
proxy.peer_group_tsr                 # Peer group TSR
proxy.net_income                     # Net income
proxy.company_selected_measure       # Company's chosen KPI
proxy.performance_measures           # All performance measures
proxy.insider_trading_policy_adopted # Governance indicator
```

## What's Next

You've learned how to parse SEC proxy statements with Python. Here are related tutorials:

- [Extract Executive Compensation from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/executive-compensation-sec-python.ipynb)
- [Track Insider Trading from SEC Form 4](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/insider-trading-sec-form4-python.ipynb)
- [Analyze 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/analyze-10k-annual-report-python.ipynb)
- [SEC EDGAR API in Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*